In [1]:
import random
import time

def draw_horizontal_boundary(fp, width):
    fp.write("+" + "-+" * width + "\n")

def draw_vertical_walls(fp, width):
    fp.write("|")
    for i in range(width - 1):
        fp.write(" " + ("|" if vertical_walls[i] == 1 else " "))
    fp.write(" |\n")

def draw_horizontal_walls(fp, width):
    fp.write("+")
    for i in range(width):
        fp.write("-+" if horizontal_walls[i] == 1 else " +")
    fp.write("\n")

def initialize_first_row(width):
    global current_set_id, last_wall_position
    random.seed(time.time())
    for i in range(width - 1):
        vertical_walls[i] = random.randint(0, 1)
        if vertical_walls[i] == 1:
            for j in range(last_wall_position + 1, i + 1):
                cell_sets[j] = current_set_id
            last_wall_position = i
            current_set_id += 1
    for i in range(last_wall_position + 1, width):
        cell_sets[i] = current_set_id
    current_set_id += 1

def place_horizontal_walls(width):
    global current_set_id
    previous_set_id = cell_sets[0]
    vertical_connection_made = 0

    for i in range(width):
        horizontal_walls[i] = random.randint(0, 1)
        if horizontal_walls[i] == 0:
            vertical_connection_made = 1

        if i < width - 1 and previous_set_id != cell_sets[i + 1]:
            if vertical_connection_made == 0:
                horizontal_walls[i] = 0
            else:
                vertical_connection_made = 0
            previous_set_id = cell_sets[i + 1]

        if i == width - 1 and vertical_connection_made == 0:
            horizontal_walls[i] = 0

        if horizontal_walls[i] == 1:
            cell_sets[i] = current_set_id
            current_set_id += 1

def connect_vertical_sets(width):
    for i in range(width - 1):
        if cell_sets[i] != cell_sets[i + 1]:
            if random.randint(0, 1) == 1:
                previous_set_id = cell_sets[i + 1]
                cell_sets[i + 1] = cell_sets[i]
                for j in range(width):
                    if cell_sets[j] == previous_set_id:
                        cell_sets[j] = cell_sets[i]
                vertical_walls[i] = 0
            else:
                vertical_walls[i] = 1
        else:
            vertical_walls[i] = 1

def finalize_vertical_connections(width):
    for i in range(width - 1):
        if cell_sets[i] != cell_sets[i + 1]:
            vertical_walls[i] = 0
            previous_set_id = cell_sets[i + 1]
            cell_sets[i + 1] = cell_sets[i]
            for j in range(width):
                if cell_sets[j] == previous_set_id:
                    cell_sets[j] = cell_sets[i]
        else:
            vertical_walls[i] = 1

if __name__ == "__main__":
    WIDTH = int(input("Enter the width of the maze: "))
    HEIGHT = int(input("Enter the height of the maze: "))

    horizontal_walls = [0] * WIDTH
    vertical_walls = [0] * (WIDTH - 1)
    cell_sets = [0] * WIDTH

    current_set_id = 0
    last_wall_position = -1

    with open("maze.maz", "wt") as fp:
        draw_horizontal_boundary(fp, WIDTH)
        initialize_first_row(WIDTH)
        draw_vertical_walls(fp, WIDTH)

        for k in range(1, HEIGHT - 1):
            place_horizontal_walls(WIDTH)
            draw_horizontal_walls(fp, WIDTH)
            connect_vertical_sets(WIDTH)
            draw_vertical_walls(fp, WIDTH)

        place_horizontal_walls(WIDTH)
        draw_horizontal_walls(fp, WIDTH)
        finalize_vertical_connections(WIDTH)
        draw_vertical_walls(fp, WIDTH)
        draw_horizontal_boundary(fp, WIDTH)


In [4]:
import random
import heapq

def create_grid(width, height):
    grid = [['+'] * (2 * width + 1) for _ in range(2 * height + 1)]
    for i in range(2 * height + 1):
        for j in range(2 * width + 1):
            if i % 2 == 0 and j % 2 == 1:
                grid[i][j] = '-'
            elif i % 2 == 1 and j % 2 == 0:
                grid[i][j] = '|'
            elif i % 2 == 1 and j % 2 == 1:
                grid[i][j] = ' '
    return grid

def display(grid):
    for row in grid:
        print("".join(row))

def save_to_file(grid, filename):
    with open(filename, 'w') as file:
        for row in grid:
            file.write("".join(row) + '\n')

def add_edges(x, y, width, height, visited, edges):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < width and 0 <= ny < height and not visited[ny][nx]:
            wall_x, wall_y = 2 * y + 1 + dy, 2 * x + 1 + dx
            weight = random.randint(1, 100)
            heapq.heappush(edges, (weight, (x, y), (nx, ny), (wall_x, wall_y)))

def generate_maze(grid, width, height):
    visited = [[False] * width for _ in range(height)]
    edges = []
    start_x, start_y = 0, 0
    visited[start_y][start_x] = True
    add_edges(start_x, start_y, width, height, visited, edges)
    
    while edges:
        _, (x, y), (nx, ny), (wall_x, wall_y) = heapq.heappop(edges)
        if not visited[ny][nx]:
            grid[wall_x][wall_y] = ' '
            visited[ny][nx] = True
            add_edges(nx, ny, width, height, visited, edges)

if __name__ == "__main__":
    width = int(input("Enter the width of the maze: "))
    height = int(input("Enter the height of the maze: "))
    
    grid = create_grid(width, height)
    generate_maze(grid, width, height)
    save_to_file(grid, 'maze.maz')
    display(grid)


+-+-+-+-+-+-+-+
|     |       |
+-+-+ +-+ + + +
| | | |   | | |
+ + + + +-+-+-+
|   |   |   | |
+ +-+ + + + + +
|     |   |   |
+-+-+-+-+-+-+-+
